# Tratamento de Dados


## 1. Importações e Carregamento dos Dados

In [29]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

# Carregar o dataset
df = pd.read_csv('BASE_1_Manga_Produtos.csv')

print(f"Dataset carregado: {df.shape[0]} linhas e {df.shape[1]} colunas")
print(f"\nValores nulos totais: {df.isnull().sum().sum()}")
print(f"\nPrimeiras linhas:")
df.head()

Dataset carregado: 1700 linhas e 15 colunas

Valores nulos totais: 1253

Primeiras linhas:


,empresa,faturamento_empresa_brl,produto,tipo_cliente,mercado_principal,indice_concorrencia_1a5,indice_valor_agregado_1a5,margem_media_percentual,indice_complexidade_produtiva_1a5,indice_capex_1a5,indice_escalabilidade_1a5,indice_exigencia_regulatoria_1a5,certificacoes_principais,indice_compatibilidade_refugo_1a5,indice_compatibilidade_operacao_fazenda_1a5
0,BRF Ingredients,5.300000e+10,Sorbet / picolé,B2B,Interno,2.0,3.5,45.66,4.0,1.0,1.0,3.0,ANVISA,2.0,1.0
1,Ambev (bebidas),7.200000e+10,Purê de manga asséptico,B2C,NaN,2.0,3.0,12.79,3.0,1.0,3.0,3.0,ANVISA,1.0,2.0
2,Bunge Brasil,9.500000e+10,Biocomposto / fertilizante orgânico,B2B,Externo,4.0,2.0,12.31,2.0,3.0,3.0,2.0,ANVISA,3.0,2.0
3,MangaVale Alimentos,4.800000e+08,Sabonete / hidratante com manga,B2B,Ambos,1.0,2.5,37.29,3.0,2.0,NaN,2.0,MAPA,2.0,2.0
4,Cutrale,3.000000e+10,Corante natural,B2B,Interno,3.0,3.5,13.45,4.0,2.0,4.0,3.0,MAPA,2.0,4.0


In [30]:
# Criar cópia para tratamento
df_tratado = df.copy()

print(f"Dataset original preservado: {df.shape[0]} linhas")
print(f"Dataset para tratamento: {df_tratado.shape[0]} linhas")

Dataset original preservado: 1700 linhas
Dataset para tratamento: 1700 linhas


## 2. Análise Inicial de Valores Faltantes

In [31]:
# Verificar valores nulos por coluna
print("=== VALORES NULOS POR COLUNA ===")
valores_nulos = df_tratado.isnull().sum()
percentual_nulos = (valores_nulos / len(df_tratado)) * 100

df_nulos = pd.DataFrame({
    'Coluna': valores_nulos.index,
    'Quantidade Nulos': valores_nulos.values,
    'Percentual (%)': percentual_nulos.values.round(2)
})
df_nulos = df_nulos[df_nulos['Quantidade Nulos'] > 0].sort_values('Quantidade Nulos', ascending=False)

print(df_nulos.to_string(index=False))
print(f"\nTotal de valores nulos: {valores_nulos.sum()}")

=== VALORES NULOS POR COLUNA ===
                                     Coluna  Quantidade Nulos  Percentual (%)
                  indice_valor_agregado_1a5                98            5.76
                                    produto                94            5.53
                                    empresa                91            5.35
                           indice_capex_1a5                91            5.35
                               tipo_cliente                90            5.29
                  indice_escalabilidade_1a5                90            5.29
          indice_compatibilidade_refugo_1a5                87            5.12
                    indice_concorrencia_1a5                84            4.94
                          mercado_principal                80            4.71
                   certificacoes_principais                79            4.65
                    margem_media_percentual                79            4.65
          indice_complexidade_p

## 3. TRATAMENTO DEFINITIVO - Ordem Obrigatória de Execução

### PASSO 1: Variáveis Categóricas

**Colunas:**
- `empresa`
- `produto`
- `tipo_cliente`
- `mercado_principal`
- `certificacoes_principais`

**Estratégia:** Criar categoria explícita "Não informado" para valores ausentes.

In [32]:
# PASSO 1: Criar categorias explícitas para variáveis categóricas
print("=== PASSO 1: VARIÁVEIS CATEGÓRICAS ===\n")

# IMPORTANTE: NÃO usar imputação por moda para variáveis categóricas
# A estratégia é criar categoria explícita "Não informado" para preservar informação
# de ausência e evitar viés de frequência que a moda introduziria.

categorical_cols = [
    "empresa",
    "produto",
    "tipo_cliente",
    "mercado_principal",
    "certificacoes_principais"
]

for col in categorical_cols:
    if col in df_tratado.columns:
        nulos_antes = df_tratado[col].isnull().sum()
        # Estratégia: categoria explícita "Não informado" (sem moda)
        df_tratado[col] = df_tratado[col].fillna("Não informado")
        nulos_depois = df_tratado[col].isnull().sum()
        print(f"{col}: {nulos_antes} valores imputados com 'Não informado' (sem moda)")
        assert nulos_depois == 0, f"Erro: ainda existem {nulos_depois} valores nulos em {col}"

print("\n✅ PASSO 1 concluído: Todas as variáveis categóricas tratadas")
print("   ✅ Nenhuma variável categórica utiliza imputação por moda")

=== PASSO 1: VARIÁVEIS CATEGÓRICAS ===

empresa: 91 valores imputados com 'Não informado' (sem moda)
produto: 94 valores imputados com 'Não informado' (sem moda)
tipo_cliente: 90 valores imputados com 'Não informado' (sem moda)
mercado_principal: 80 valores imputados com 'Não informado' (sem moda)
certificacoes_principais: 79 valores imputados com 'Não informado' (sem moda)

✅ PASSO 1 concluído: Todas as variáveis categóricas tratadas
   ✅ Nenhuma variável categórica utiliza imputação por moda


### 📌 PASSO 2: Variáveis Numéricas Sensíveis (MNAR)

**Colunas:**
- `faturamento_empresa_brl`
- `margem_media_percentual`

**Justificativa:**
- São variáveis financeiras
- Têm alta probabilidade de ausência relacionada ao próprio valor
- Não podem ser tratadas como MCAR, mesmo que testes não rejeitem H₀

**Estratégia:**
1. Criar flags de missing
2. Imputação conservadora por mediana condicional (por `tipo_cliente`)

In [33]:
# PASSO 2: Criar flags de missing para variáveis MNAR
print("=== PASSO 2: VARIÁVEIS NUMÉRICAS SENSÍVEIS (MNAR) ===\n")

# IMPORTANTE: Flags de missing para variáveis com possível caráter MNAR
# Estas variáveis financeiras (faturamento_empresa_brl e margem_media_percentual) 
# têm alta probabilidade de ausência relacionada ao próprio valor não observado.
# A ausência pode indicar confidencialidade, valores extremos, ou decisão estratégica.
# As flags preservam essa informação para análises futuras e enriquecem o modelo.
# MNAR não é testável diretamente, mas o contexto econômico sugere essa classificação.

# Criar flags ANTES da imputação
df_tratado["faturamento_missing"] = df_tratado["faturamento_empresa_brl"].isna().astype(int)
df_tratado["margem_missing"] = df_tratado["margem_media_percentual"].isna().astype(int)

print(f"Flags criadas:")
print(f"  faturamento_missing: {df_tratado['faturamento_missing'].sum()} valores ausentes marcados")
print(f"  margem_missing: {df_tratado['margem_missing'].sum()} valores ausentes marcados")
print(f"\n📌 Nota: Flags preservam informação sobre ausências possivelmente MNAR")

print("\n✅ Flags de missing criadas")

=== PASSO 2: VARIÁVEIS NUMÉRICAS SENSÍVEIS (MNAR) ===

Flags criadas:
  faturamento_missing: 76 valores ausentes marcados
  margem_missing: 79 valores ausentes marcados

📌 Nota: Flags preservam informação sobre ausências possivelmente MNAR

✅ Flags de missing criadas


In [34]:
# PASSO 2 (continuação): Imputação conservadora por mediana condicional
print("\n=== IMPUTAÇÃO POR MEDIANA CONDICIONAL (por tipo_cliente) ===\n")

# Verificar se tipo_cliente tem valores nulos (deve ter sido tratado no PASSO 1)
if df_tratado["tipo_cliente"].isnull().sum() > 0:
    print("⚠️ ATENÇÃO: tipo_cliente ainda tem valores nulos. Tratando primeiro...")
    df_tratado["tipo_cliente"] = df_tratado["tipo_cliente"].fillna("Não informado")

# Imputar faturamento_empresa_brl por mediana condicional
nulos_faturamento_antes = df_tratado["faturamento_empresa_brl"].isnull().sum()
df_tratado["faturamento_empresa_brl"] = df_tratado.groupby("tipo_cliente")["faturamento_empresa_brl"].transform(
    lambda x: x.fillna(x.median())
)
nulos_faturamento_depois = df_tratado["faturamento_empresa_brl"].isnull().sum()

print(f"faturamento_empresa_brl:")
print(f"  Nulos antes: {nulos_faturamento_antes}")
print(f"  Nulos depois: {nulos_faturamento_depois}")

# Se ainda houver nulos (grupos sem valores), usar mediana global como fallback
if nulos_faturamento_depois > 0:
    print(f"  ⚠️ Usando mediana global como fallback para {nulos_faturamento_depois} valores")
    mediana_global = df_tratado["faturamento_empresa_brl"].median()
    df_tratado["faturamento_empresa_brl"] = df_tratado["faturamento_empresa_brl"].fillna(mediana_global)

# Imputar margem_media_percentual por mediana condicional
nulos_margem_antes = df_tratado["margem_media_percentual"].isnull().sum()
df_tratado["margem_media_percentual"] = df_tratado.groupby("tipo_cliente")["margem_media_percentual"].transform(
    lambda x: x.fillna(x.median())
)
nulos_margem_depois = df_tratado["margem_media_percentual"].isnull().sum()

print(f"\nmargem_media_percentual:")
print(f"  Nulos antes: {nulos_margem_antes}")
print(f"  Nulos depois: {nulos_margem_depois}")

# Se ainda houver nulos (grupos sem valores), usar mediana global como fallback
if nulos_margem_depois > 0:
    print(f"  ⚠️ Usando mediana global como fallback para {nulos_margem_depois} valores")
    mediana_global = df_tratado["margem_media_percentual"].median()
    df_tratado["margem_media_percentual"] = df_tratado["margem_media_percentual"].fillna(mediana_global)

# Garantir que margem_media_percentual esteja no intervalo [0, 100]
# Após imputação, valores podem estar fora da faixa válida (percentual)
df_tratado["margem_media_percentual"] = df_tratado["margem_media_percentual"].clip(lower=0, upper=100)
print(f"  ✅ Margem garantida no intervalo [0, 100] após imputação")

print("\n✅ PASSO 2 concluído: Variáveis MNAR tratadas com flags e imputação condicional")


=== IMPUTAÇÃO POR MEDIANA CONDICIONAL (por tipo_cliente) ===

faturamento_empresa_brl:
  Nulos antes: 76
  Nulos depois: 0

margem_media_percentual:
  Nulos antes: 79
  Nulos depois: 0
  ✅ Margem garantida no intervalo [0, 100] após imputação

✅ PASSO 2 concluído: Variáveis MNAR tratadas com flags e imputação condicional


### 📌 PASSO 3: Variáveis Numéricas Ordinais (1 a 5)

**Colunas:**
- `indice_concorrencia_1a5`
- `indice_valor_agregado_1a5`
- `indice_complexidade_produtiva_1a5`
- `indice_capex_1a5`
- `indice_escalabilidade_1a5`
- `indice_exigencia_regulatoria_1a5`
- `indice_compatibilidade_refugo_1a5`
- `indice_compatibilidade_operacao_fazenda_1a5`

**Classificação:** MAR (Missing At Random)

**Estratégia:** Imputação por mediana condicional por `produto`

In [35]:
# PASSO 3: Imputação de variáveis ordinais por mediana condicional
print("=== PASSO 3: VARIÁVEIS ORDINAIS (1-5) ===\n")

ordinal_cols = [
    "indice_concorrencia_1a5",
    "indice_valor_agregado_1a5",
    "indice_complexidade_produtiva_1a5",
    "indice_capex_1a5",
    "indice_escalabilidade_1a5",
    "indice_exigencia_regulatoria_1a5",
    "indice_compatibilidade_refugo_1a5",
    "indice_compatibilidade_operacao_fazenda_1a5"
]

# Verificar se produto tem valores nulos (deve ter sido tratado no PASSO 1)
if df_tratado["produto"].isnull().sum() > 0:
    print("⚠️ ATENÇÃO: produto ainda tem valores nulos. Tratando primeiro...")
    df_tratado["produto"] = df_tratado["produto"].fillna("Não informado")

for col in ordinal_cols:
    if col in df_tratado.columns:
        nulos_antes = df_tratado[col].isnull().sum()
        
        # Imputação por mediana condicional por produto
        df_tratado[col] = df_tratado.groupby("produto")[col].transform(
            lambda x: x.fillna(x.median())
        )
        
        nulos_depois = df_tratado[col].isnull().sum()
        
        # Se ainda houver nulos (produtos sem valores), usar mediana global como fallback
        if nulos_depois > 0:
            mediana_global = df_tratado[col].median()
            df_tratado[col] = df_tratado[col].fillna(mediana_global)
            nulos_depois_final = df_tratado[col].isnull().sum()
            print(f"{col}: {nulos_antes} → {nulos_depois} → {nulos_depois_final} (com fallback global)")
        else:
            print(f"{col}: {nulos_antes} valores imputados por mediana condicional")
        
        # AJUSTE OBRIGATÓRIO: Garantir coerência ordinal após imputação
        # Aplicar round() para valores inteiros e clip(1,5) para garantir faixa válida
        # Isso garante que valores imputados respeitem a escala ordinal de 1 a 5
        df_tratado[col] = df_tratado[col].round().clip(lower=1, upper=5)

print("\n✅ PASSO 3 concluído: Todas as variáveis ordinais tratadas")
print("   ✅ Valores ordinais arredondados e garantidos na faixa [1, 5]")

=== PASSO 3: VARIÁVEIS ORDINAIS (1-5) ===

indice_concorrencia_1a5: 84 valores imputados por mediana condicional
indice_valor_agregado_1a5: 98 valores imputados por mediana condicional
indice_complexidade_produtiva_1a5: 78 valores imputados por mediana condicional


indice_capex_1a5: 91 valores imputados por mediana condicional
indice_escalabilidade_1a5: 90 valores imputados por mediana condicional
indice_exigencia_regulatoria_1a5: 69 valores imputados por mediana condicional
indice_compatibilidade_refugo_1a5: 87 valores imputados por mediana condicional
indice_compatibilidade_operacao_fazenda_1a5: 67 valores imputados por mediana condicional

✅ PASSO 3 concluído: Todas as variáveis ordinais tratadas
   ✅ Valores ordinais arredondados e garantidos na faixa [1, 5]


## 4. Verificações Finais (Obrigatórias)

In [36]:
# Verificação 1: Nenhuma coluna deve permanecer com NaN
print("=== VERIFICAÇÃO 1: VALORES NULOS RESTANTES ===\n")

nulos_restantes = df_tratado.isnull().sum()
colunas_com_nulos = nulos_restantes[nulos_restantes > 0]

if len(colunas_com_nulos) == 0:
    print("✅ Nenhuma coluna possui valores nulos")
    print(f"   Total de valores nulos: {df_tratado.isnull().sum().sum()}")
else:
    print("⚠️ ATENÇÃO: Ainda existem valores nulos nas seguintes colunas:")
    print(colunas_com_nulos)

# Assert obrigatório
assert df_tratado.isnull().sum().sum() == 0, "ERRO: Ainda existem valores nulos no dataset!"
print("\n✅ Assert passou: Nenhum valor nulo restante")

=== VERIFICAÇÃO 1: VALORES NULOS RESTANTES ===

✅ Nenhuma coluna possui valores nulos
   Total de valores nulos: 0

✅ Assert passou: Nenhum valor nulo restante


In [37]:
# Verificação 2: Flags devem existir
print("=== VERIFICAÇÃO 2: FLAGS DE MISSING ===\n")

flags_obrigatorias = ["faturamento_missing", "margem_missing"]

for flag in flags_obrigatorias:
    if flag in df_tratado.columns:
        print(f"✅ {flag}: existe ({df_tratado[flag].sum()} valores ausentes marcados)")
    else:
        print(f"❌ ERRO: {flag} não existe no dataset!")

# Asserts obrigatórios
assert "faturamento_missing" in df_tratado.columns, "ERRO: Flag faturamento_missing não existe!"
assert "margem_missing" in df_tratado.columns, "ERRO: Flag margem_missing não existe!"

print("\n✅ Assert passou: Todas as flags obrigatórias existem")

=== VERIFICAÇÃO 2: FLAGS DE MISSING ===

✅ faturamento_missing: existe (76 valores ausentes marcados)
✅ margem_missing: existe (79 valores ausentes marcados)

✅ Assert passou: Todas as flags obrigatórias existem


In [38]:
# Verificação 3: Validação de faixas de valores
print("=== VERIFICAÇÃO 3: VALIDAÇÃO DE FAIXAS DE VALORES ===\n")

# Índices devem estar entre 1-5
ordinal_cols = [
    "indice_concorrencia_1a5",
    "indice_valor_agregado_1a5",
    "indice_complexidade_produtiva_1a5",
    "indice_capex_1a5",
    "indice_escalabilidade_1a5",
    "indice_exigencia_regulatoria_1a5",
    "indice_compatibilidade_refugo_1a5",
    "indice_compatibilidade_operacao_fazenda_1a5"
]

erros_faixa = []
for col in ordinal_cols:
    if col in df_tratado.columns:
        valores_fora_faixa = df_tratado[(df_tratado[col] < 1) | (df_tratado[col] > 5)][col]
        if len(valores_fora_faixa) > 0:
            print(f"⚠️ {col}: {len(valores_fora_faixa)} valores fora da faixa 1-5")
            erros_faixa.append(col)
        else:
            print(f"✅ {col}: Todos os valores na faixa 1-5")

# Margem deve estar entre 0-100
if 'margem_media_percentual' in df_tratado.columns:
    margem_invalida = df_tratado[(df_tratado['margem_media_percentual'] < 0) | 
                                (df_tratado['margem_media_percentual'] > 100)]['margem_media_percentual']
    if len(margem_invalida) > 0:
        print(f"⚠️ margem_media_percentual: {len(margem_invalida)} valores fora da faixa 0-100")
        erros_faixa.append('margem_media_percentual')
    else:
        print(f"✅ margem_media_percentual: Todos os valores na faixa 0-100")

# Faturamento deve ser positivo
if 'faturamento_empresa_brl' in df_tratado.columns:
    faturamento_negativo = df_tratado[df_tratado['faturamento_empresa_brl'] < 0]['faturamento_empresa_brl']
    if len(faturamento_negativo) > 0:
        print(f"⚠️ faturamento_empresa_brl: {len(faturamento_negativo)} valores negativos")
        erros_faixa.append('faturamento_empresa_brl')
    else:
        print(f"✅ faturamento_empresa_brl: Todos os valores são positivos")

if len(erros_faixa) == 0:
    print("\n✅ Todas as faixas de valores estão corretas")
else:
    print(f"\n⚠️ {len(erros_faixa)} coluna(s) com valores fora da faixa esperada")

=== VERIFICAÇÃO 3: VALIDAÇÃO DE FAIXAS DE VALORES ===

✅ indice_concorrencia_1a5: Todos os valores na faixa 1-5
✅ indice_valor_agregado_1a5: Todos os valores na faixa 1-5
✅ indice_complexidade_produtiva_1a5: Todos os valores na faixa 1-5
✅ indice_capex_1a5: Todos os valores na faixa 1-5
✅ indice_escalabilidade_1a5: Todos os valores na faixa 1-5
✅ indice_exigencia_regulatoria_1a5: Todos os valores na faixa 1-5
✅ indice_compatibilidade_refugo_1a5: Todos os valores na faixa 1-5
✅ indice_compatibilidade_operacao_fazenda_1a5: Todos os valores na faixa 1-5
✅ margem_media_percentual: Todos os valores na faixa 0-100
✅ faturamento_empresa_brl: Todos os valores são positivos

✅ Todas as faixas de valores estão corretas


## 5. Resumo do Tratamento

In [39]:
# Resumo final
print("=== RESUMO DO TRATAMENTO ===\n")

print(f"Dataset original: {df.shape[0]} linhas x {df.shape[1]} colunas")
print(f"Dataset tratado: {df_tratado.shape[0]} linhas x {df_tratado.shape[1]} colunas")
print(f"  (Adicionadas {df_tratado.shape[1] - df.shape[1]} colunas de flags)")

print(f"\nValores nulos original: {df.isnull().sum().sum()}")
print(f"Valores nulos após tratamento: {df_tratado.isnull().sum().sum()}")

print("\n=== ESTRATÉGIAS APLICADAS ===")
print("1. Variáveis Categóricas: Categoria 'Não informado' (sem moda)")
print("2. Variáveis MNAR (faturamento, margem): Flags + Mediana condicional por tipo_cliente")
print("3. Variáveis Ordinais (índices 1-5): Mediana condicional por produto")

print("\n=== FLAGS CRIADAS ===")
print(f"  faturamento_missing: {df_tratado['faturamento_missing'].sum()} valores ausentes")
print(f"  margem_missing: {df_tratado['margem_missing'].sum()} valores ausentes")

print("\n✅ Tratamento concluído com sucesso!")

=== RESUMO DO TRATAMENTO ===

Dataset original: 1700 linhas x 15 colunas
Dataset tratado: 1700 linhas x 17 colunas
  (Adicionadas 2 colunas de flags)

Valores nulos original: 1253
Valores nulos após tratamento: 0

=== ESTRATÉGIAS APLICADAS ===
1. Variáveis Categóricas: Categoria 'Não informado' (sem moda)
2. Variáveis MNAR (faturamento, margem): Flags + Mediana condicional por tipo_cliente
3. Variáveis Ordinais (índices 1-5): Mediana condicional por produto

=== FLAGS CRIADAS ===
  faturamento_missing: 76 valores ausentes
  margem_missing: 79 valores ausentes

✅ Tratamento concluído com sucesso!


In [40]:
# Visualizar primeiras linhas do dataset tratado
print("=== PRIMEIRAS LINHAS DO DATASET TRATADO ===")
df_tratado.head(10)

=== PRIMEIRAS LINHAS DO DATASET TRATADO ===


,empresa,faturamento_empresa_brl,produto,tipo_cliente,mercado_principal,indice_concorrencia_1a5,indice_valor_agregado_1a5,margem_media_percentual,indice_complexidade_produtiva_1a5,indice_capex_1a5,indice_escalabilidade_1a5,indice_exigencia_regulatoria_1a5,certificacoes_principais,indice_compatibilidade_refugo_1a5,indice_compatibilidade_operacao_fazenda_1a5,faturamento_missing,margem_missing
0,BRF Ingredients,5.300000e+10,Sorbet / picolé,B2B,Interno,2.0,4.0,45.660,4.0,1.0,1.0,3.0,ANVISA,2.0,1.0,0,0
1,Ambev (bebidas),7.200000e+10,Purê de manga asséptico,B2C,Não informado,2.0,3.0,12.790,3.0,1.0,3.0,3.0,ANVISA,1.0,2.0,0,0
2,Bunge Brasil,9.500000e+10,Biocomposto / fertilizante orgânico,B2B,Externo,4.0,2.0,12.310,2.0,3.0,3.0,2.0,ANVISA,3.0,2.0,0,0
3,MangaVale Alimentos,4.800000e+08,Sabonete / hidratante com manga,B2B,Ambos,1.0,2.0,37.290,3.0,2.0,3.0,2.0,MAPA,2.0,2.0,0,0
4,Cutrale,3.000000e+10,Corante natural,B2B,Interno,3.0,4.0,13.450,4.0,2.0,4.0,3.0,MAPA,2.0,4.0,0,0
5,AgroDoce Nordeste,2.200000e+08,Óleo / manteiga de semente de manga,B2B,Externo,3.0,4.0,31.020,3.0,4.0,1.0,5.0,Internacional,3.0,3.0,0,0
6,VerdePulp Brasil,3.100000e+08,Não informado,B2B,Interno,2.0,4.0,18.220,4.0,1.0,1.0,3.0,Nenhuma,3.0,1.0,0,0
7,NordFruit Export,5.400000e+08,Ração / ingrediente para ração animal,Não informado,Interno,2.0,4.0,43.970,3.0,4.0,3.0,4.0,Internacional,2.0,4.0,0,0
8,Cutrale,3.000000e+10,Base para néctar,B2B,Interno,3.0,2.0,19.200,2.0,3.0,3.0,3.0,Internacional,2.0,3.0,0,0
9,Fazenda Sol do Vale,1.800000e+08,Óleo / manteiga de semente de manga,B2C,Externo,3.0,4.0,28.415,4.0,3.0,3.0,3.0,MAPA,2.0,2.0,0,1


In [41]:
# Estatísticas descritivas das variáveis tratadas
print("=== ESTATÍSTICAS DESCRITIVAS (VARIÁVEIS NUMÉRICAS) ===")
df_tratado.select_dtypes(include=[np.number]).describe()

=== ESTATÍSTICAS DESCRITIVAS (VARIÁVEIS NUMÉRICAS) ===


,faturamento_empresa_brl,indice_concorrencia_1a5,indice_valor_agregado_1a5,margem_media_percentual,indice_complexidade_produtiva_1a5,indice_capex_1a5,indice_escalabilidade_1a5,indice_exigencia_regulatoria_1a5,indice_compatibilidade_refugo_1a5,indice_compatibilidade_operacao_fazenda_1a5,faturamento_missing,margem_missing
count,1.700000e+03,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000
mean,3.930561e+10,2.984118,2.877647,28.331885,3.041765,2.824118,3.037059,2.767059,2.288235,2.568824,0.044706,0.046471
std,7.956328e+10,1.115088,0.882953,10.458153,1.094702,1.197508,1.109291,1.164957,1.227688,1.242505,0.206718,0.210564
min,1.200000e+08,1.000000,1.000000,7.390000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,4.800000e+08,2.000000,2.000000,19.350000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,0.000000,0.000000
50%,4.200000e+09,3.000000,3.000000,27.690000,3.000000,3.000000,3.000000,3.000000,2.000000,2.000000,0.000000,0.000000
75%,5.300000e+10,4.000000,4.000000,36.890000,4.000000,4.000000,4.000000,3.000000,3.000000,3.000000,0.000000,0.000000
max,3.750000e+11,5.000000,5.000000,54.700000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1.000000,1.000000


## 6. Exportação do Dataset Tratado (Opcional)

In [42]:
# Exportar dataset tratado para CSV
# Descomente as linhas abaixo para exportar

# df_tratado.to_csv('BASE_1_Manga_Produtos_TRATADO.csv', index=False)
# print("Dataset tratado exportado para 'BASE_1_Manga_Produtos_TRATADO.csv'")